# Pytorch C++ Frontend

If you are an enthusiast/intermediate/expert in ML/AI/DeepLearning , you will already know something (or lot of things) about Pytorch. In this blog we take a step back and discuss and learn about different ways or frontends which you can use to build the Pytorch model. To be more speicific, we focus our attention on using Pytorch's C++ frontend capabilities.

In this blog we try 100% hands on approach to get to the bottom of how things can be done using C++ frontend. Here is the official blog which will explain why one would need the C++ models than the python models. 

Here is some general caveat before we dive in.

* We assume reader has atleaset some basic programming knowledge in C/C++ and python.
* This blog is **NOT** a C++ language tutorial. But rather a tutorial which explains how to use or implement the pytorch models using C++.

Here is a quick peek at what to expect out of this blog and upcoming blogs. 

I try to break it in to 4 different logical parts

* Inital setup and building the C++ code.
* Weights-Biases and  Perceptrons from scratch. Using Pytorch Tensors.
* MNIST from simple Perceptrons.
* Implement a CNN for CIFAR-10 dataset.

## Initial setup.

To work with C++ frontend we need the pytorch libraries. Let us see how to [install](https://pytorch.org/cppdocs/installing.html) the same.

* First download the libraries

```
wget https://download.pytorch.org/libtorch/nightly/cpu/libtorch-shared-with-deps-latest.zip

```

* Extract the libs

```
unzip libtorch-shared-with-deps-latest.zip -d /opt/pytorch
```

Notice that we are extracting the libs to the path **/opt/pytorch/**. This is just for our convenience. You can extract it where ever you want. This approach has some slight advantage, which we see later.

That's it it is as simple as that. Now let us try to do some basic operations in C++.

**NOTE: You can also build the libpytorch libs from the source and install. Which we will not cover here.**


Let us use this small piece of code written is python to understand and build our model using C++ frontend.

### Tensor creation using Pytorch 

### Python frontend:


```python
# First, import PyTorch
import torch

### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 5 random normal variables
features = torch.randn((1, 5))
print(features)

############# OUTPUT #################

tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])

```

Now let us write the equivalent code in C++ and compile and check our results

### C++ frontend:
```cpp
// main.cpp
#include <torch/torch.h>
#include <ATen/Context.h>
#include <iostream>

int main()
{
  // Seed for Random number
  torch::manual_seed(9);
  torch::Tensor features = torch::rand({1, 5});
  std::cout << "features = " << features << "\n";
}

################ OUTPUT #######################

features =  0.6558  0.3020  0.4799  0.7774  0.9180
[ Variable[CPUFloatType]{1,5} ]

```


If you already understood the above code great job! You can skip this section and move to next one where we discuss about compiling and linking the pytorch libs to generate a binary.

For those of you who need some more help. Let us break down the code line by line

As you all know we just import the python module as follows.

```python
import torch
```

Similarly in C++ we use the include statement as follows to include the required headers. Which is similar to **import** in python

```cpp
#include <torch/torch.h>
#include <ATen/Context.h>
```

Now, we seed our randon number in python as follows.

```python
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable
```

Similarly we seed the randon number geneator as follows in C++.
```cpp
  // Seed for Random number
  torch::manual_seed(9);
```


Now let us generate a **Tensor** of one-Dimention. 
```python
# Features are 5 random normal variables
features = torch.randn((1, 5))
print(features)
```

This is how you generate a **Tensor** of one-Dimention using C++ code. And as usual we use **std::cout** to print the results to console.
```cpp
  torch::Tensor features = torch::rand({1, 5});
  std::cout << "features = " << features << "\n";

```

As we can see there is lot of similarities in the way python code is written and the C++ code is written. This is mainly because the authors of pytorch lib has gone an extra mile to keep the syntax as simple and similar as possible to python, so that the people from python and C++ background can work seemlessly. It is one of the **[philosophy](https://pytorch.org/cppdocs/frontend.html)** behind the pytorh C++ frontend development.


**If you observe carfully, if you just copy paste the python code and replace all the **"."** with **"::"** you have an almost working C++ code!**

**NOTE:** Please note that it is an **"almost working"** C++ code. We will visit on this topic soon. Till then just assume you can always copy paste the python code and replace the "." with "::" to get going with your C++ migration of python code.

## Building C++ code using Cmake.

### Create CMakeLists.txt file  

C++ code can be easily integrated with the pytorch libs using **[Cmake](https://cmake.org/)**. Let us see how to write one for our example. 

```cmake
CMAKE_MINIMUM_REQUIRED(VERSION 3.0 FATAL_ERROR)

PROJECT(HelloPytorch) 
    
SET(CMAKE_CXX_STANDARD 11)
SET(TORCH_DIR /opt/pytorch/)
    
FIND_PACKAGE(Torch REQUIRED)
    
INCLUDE_DIRECTORIES(${TORCH_INCLUDE_DIRS})

ADD_EXECUTABLE(HelloWorld main.cpp)
TARGET_LINK_LIBRARIES(HelloWolrd "${TORCH_LIBRARIES}")

```

If you understand the Cmake syntax, you can skip over to the next section.

Now let us try to understand what is happening with our **CMakeLists.txt** file and how everything is put together.

We specify the minimum version of Cmake required.
```cmake
CMAKE_MINIMUM_REQUIRED(VERSION 3.0 FATAL_ERROR)
```

Let us give some name to our project.
```cmake
PROJECT(HelloPytorch) 
```

libPytorch requires you to compile to code using compiler which supports C++11 or greater. This is how we let the compiler to enable the **--std=c++11** flag during compilation.
```cmake
SET(CMAKE_CXX_STANDARD 11)
```

This one is optional. Initially we unzipped the libpytorch.zip file in to **"/opt/pytorch"** dirctory. We are hinting the Cmake about where to look for the pytorch dependencies. Ideally Cmake should be able to find the packages by itself. If the compilation is failing then it is good idea to set this flag to help the Cmake to identify where and which version of libpytorch to find.
```cmake
SET(TORCH_DIR /opt/pytorch/)
```

The following line does all the magic of finding the required dependencies for our project related to libpytorch.
```cmake
FIND_PACKAGE(Torch REQUIRED)
```

Include the pytorch directories.
```cmake
INCLUDE_DIRECTORIES(${TORCH_INCLUDE_DIRS})
```

Let us create an executable named "HelloWorld" from our main.cpp file.

```cmake
ADD_EXECUTABLE(HelloWorld main.cpp)
```

This is how we let the compiler know which libs to use to link our binary. In this case **libpytorch.so**
```cmake
TARGET_LINK_LIBRARIES(HelloWolrd "${TORCH_LIBRARIES}")
```

### Build

Here is the list of commands to be executed to generate the binary.

```bash
mkdir build
cd build
cmake ..
make
```

If the build goes fine new find a binary named **HelloWorld** being generated in **build** directory


## Output

### Python
```python
tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])
```

### C++
```cpp

./HelloWorld

features =  0.6558  0.3020  0.4799  0.7774  0.9180
[ Variable[CPUFloatType]{1,5} ]

```
Since we are generating random numbers it is expected to be different. There are different ways to print a.k.a **"pretty prints"** the tensor using C++, which we will discuss as and when we find the need to use one.